In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch.nn import Linear, LSTM
from torch.optim import Adam
from matplotlib import pyplot as plt

In [3]:
import skorch

In [4]:
from skorch import NeuralNetRegressor

## Read Available Data

In [6]:
offStat = pd.read_csv('./data/stats_offshore.csv', index_col = 0, parse_dates = True)

In [7]:
gcStat = pd.read_csv('./data/stats_GC.csv', index_col = 0, parse_dates = True)

In [8]:
gcStat['target_hs'] =  offStat['hs']

In [9]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [10]:
?model.fit

Object `model.fit` not found.


In [29]:

# Split the data into train and test sets
train_data = gcStat.loc[gcStat.index <= '2018-01-01']
test_data = gcStat.loc[gcStat.index > '2018-01-01']

# Define the hyperparameters to tune
hyperparameters = {
    'batch_size': [128, 256, 512],
    'max_epochs': [200, 250, 300],
    'lr': [0.001, 0.0001, 0.00001]
}

# Create the model
class LSTMRegressor(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = LSTM(input_size, hidden_size)
        self.linear = Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.shape[0], x.shape[1], 1)
        x = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear(x)
        return x

model = NeuralNetRegressor(
    module=LSTMRegressor,
    max_epochs=150,
    batch_size=10,
    optimizer = Adam,
    criterion = torch.nn.MSELoss(),
    module__n_layers=2
)
    
    
    
# Create the model
#model = LSTMRegressor(train_data.shape[1], 128, 1)

# Compile the model
criterion = torch.nn.MSELoss()
#model.train()

# Perform hyperparameter tuning
early_stopping = EarlyStopping(patience=5)
grid_search = GridSearchCV(model, hyperparameters, cv=5, verbose=1, scoring='neg_mean_squared_error')
grid_search.fit(train_data['hs'],y = train_data['target_hs'], epochs=100, callbacks=[early_stopping], error_score='raise')

# Get the best model
best_model = grid_search.best_estimator_

# Plot the training performance
plt.plot(grid_search.cv_results_['mean_test_score'], label='Training performance')
plt.xlabel('Number of iterations')
plt.ylabel('Mean squared error')
plt.legend()
plt.show()

# Make predictions
predictions = best_model.predict(test_data)

# Plot the results
plt.plot(test_data['Value'], label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.show()

Fitting 5 folds for each of 27 candidates, totalling 135 fits


ValueError: 
All the 135 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Leo\anaconda3\envs\wavespec\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Leo\anaconda3\envs\wavespec\lib\site-packages\skorch\regressor.py", line 91, in fit
    return super(NeuralNetRegressor, self).fit(X, y, **fit_params)
  File "C:\Users\Leo\anaconda3\envs\wavespec\lib\site-packages\skorch\net.py", line 1300, in fit
    self.initialize()
  File "C:\Users\Leo\anaconda3\envs\wavespec\lib\site-packages\skorch\net.py", line 886, in initialize
    self._initialize_module()
  File "C:\Users\Leo\anaconda3\envs\wavespe

In [20]:
model.

<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class '__main__.LSTMRegressor'>,
  module__n_layers=2,
)